# Amusement avec les images : Masques et compositions

Ce chapitre explore des techniques avancées de manipulation d'images en Python. Nous apprendrons à :
-   Charger et redimensionner des images
-   Créer des masques basés sur les canaux de couleur (technique du "green screen")
-   Appliquer des opérations morphologiques pour affiner les masques
-   Composer plusieurs images ensemble
-   Appliquer des filtres de flou de manière appropriée

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
from scipy.ndimage import gaussian_filter, binary_closing, binary_opening, binary_erosion, binary_dilation
from skimage import io, transform
import matplotlib.pyplot as plt

---

## 1. Chargement et prétraitement des images

Avant de combiner des images, nous devons les préparer :
-   **Chargement** : Utilisation de `io.imread()` pour lire les fichiers
-   **Inspection** : Vérification des dimensions et des plages de valeurs
-   **Recadrage (Cropping)** : Extraction d'une région d'intérêt avec le slicing
-   **Redimensionnement** : Ajustement de la taille avec `transform.resize()`

L'objectif est d'obtenir deux images de dimensions compatibles pour la composition.

In [ ]:
# Chargement des deux images depuis les fichiers
just_do_it = io.imread('just_do_it.jpg')
fire = io.imread('fire.jpg')

# Affichage des dimensions et des plages de valeurs
print(just_do_it.shape, fire.shape)
print(np.min(just_do_it), np.max(just_do_it))
print(np.min(fire), np.max(fire))

# Recadrage de la première image pour isoler la région d'intérêt
# Syntaxe : [lignes_début:lignes_fin, colonnes_début:colonnes_fin]
fig, axs = plt.subplots(1, 2, figsize=(16, 4))
axs[0].imshow(just_do_it)
just_do_it_crop = just_do_it[0:800, 400:1200]
axs[1].imshow(just_do_it_crop)
plt.show()

# Redimensionnement de la seconde image pour correspondre aux dimensions
# transform.resize() retourne des valeurs entre 0 et 1, on les remet en 0-255
fig, axs = plt.subplots(1, 2, figsize=(16, 4))
axs[0].imshow(fire)
fire_resize = (transform.resize(fire, (800,800)) * 255).astype(np.uint8)
axs[1].imshow(fire_resize)
plt.show()

# Vérification que les dimensions correspondent maintenant
print(just_do_it_crop.shape, fire_resize.shape)

---

## 2. Exploration des canaux RGB

Pour créer un masque efficace, nous devons identifier quel canal de couleur nous permettra de distinguer le sujet de l'arrière-plan :
-   **Canal Rouge** : Utile pour les objets rouges
-   **Canal Vert** : Idéal pour les "green screens" (fonds verts)
-   **Canal Bleu** : Utile pour les ciels ou objets bleus

Ici, nous recherchons un fond vert uniforme pour extraire le sujet.

In [ ]:
# Visualisation des trois canaux RGB séparément
fig, axs = plt.subplots(1, 4, figsize=(16, 4))
im = axs[0].imshow(just_do_it_crop)

# Canal Rouge uniquement
tmp_arr = just_do_it_crop.copy()
tmp_arr[:,:,1:3] = 0  # Met Vert et Bleu à zéro
im = axs[1].imshow(tmp_arr)

# Canal Vert uniquement
tmp_arr = just_do_it_crop.copy()
tmp_arr[:,:,0:3:2] = 0  # Met Rouge et Bleu à zéro
im = axs[2].imshow(tmp_arr)

# Canal Bleu uniquement
tmp_arr = just_do_it_crop.copy()
tmp_arr[:,:,0:2] = 0  # Met Rouge et Vert à zéro
im = axs[3].imshow(tmp_arr)

fig.tight_layout()
plt.show()

---

## 3. Création de masques par seuillage

Le seuillage (thresholding) permet de créer un masque binaire :
-   **Valeurs < seuil** : Mises à 0 (noir)
-   **Valeurs ≥ seuil** : Mises à 255 (blanc)

Cette technique, particulièrement efficace avec le canal vert pour un "green screen", permet de séparer le sujet de l'arrière-plan.

In [ ]:
# Application d'un seuil à chaque canal pour identifier le fond vert
fig, axs = plt.subplots(1, 4, figsize=(16, 4))
im = axs[0].imshow(just_do_it_crop)

# Seuillage du canal Rouge
tmp_arr = just_do_it_crop.copy()
tmp_arr[:,:,1:3] = 0
tmp_arr[tmp_arr < 200] = 0
tmp_arr[tmp_arr >= 200] = 255
im = axs[1].imshow(tmp_arr)

# Seuillage du canal Vert - c'est celui qui capture le mieux le fond
tmp_arr = just_do_it_crop.copy()
tmp_arr[:,:,0:3:2] = 0
tmp_arr[tmp_arr < 200] = 0
tmp_arr[tmp_arr >= 200] = 255
im = axs[2].imshow(tmp_arr)

# Seuillage du canal Bleu
tmp_arr = just_do_it_crop.copy()
tmp_arr[:,:,0:2] = 0
tmp_arr[tmp_arr < 200] = 0
tmp_arr[tmp_arr >= 200] = 255
im = axs[3].imshow(tmp_arr)

fig.tight_layout()
plt.show()

---

## 4. Opérations morphologiques pour affiner le masque

Les opérations morphologiques permettent d'améliorer la qualité du masque :
-   **`binary_opening()`** : Élimine les petits trous et bruits (érosion puis dilatation)
-   **`binary_dilation()`** : Agrandit les régions blanches
-   **`binary_erosion()`** : Réduit les régions blanches
-   **`np.invert()`** : Inverse le masque (noir devient blanc et vice versa)

Ces opérations permettent d'obtenir des contours plus nets et réguliers.

In [ ]:
# Génération du masque à partir du canal vert avec opérations morphologiques
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
im = axs[0].imshow(just_do_it_crop)

# Extraction et seuillage du canal vert
tmp_arr = just_do_it_crop.copy()
tmp_arr[:,:,0:3:2] = 0
tmp_arr[tmp_arr < 200] = 0
tmp_arr[tmp_arr >= 200] = 255

# Somme sur les canaux et application d'opérations morphologiques
# opening : élimine le bruit, dilation : agrandit le masque
mask = np.sum(tmp_arr, axis=-1)
mask = binary_dilation(binary_opening(mask, iterations=2), iterations=2)
im = axs[1].imshow(mask)

# Création du masque inversé pour le sujet (érosion pour affiner les bords)
inverted_mask = binary_erosion(np.invert(mask), iterations=1)
print(inverted_mask.shape)
im = axs[2].imshow(inverted_mask)

fig.tight_layout()
plt.show()

---

## 5. Application des masques aux images

Les masques permettent de sélectionner des régions d'une image :
-   **Multiplication par le masque** : Conserve uniquement les pixels où le masque est actif (1)
-   **`np.expand_dims()`** : Ajoute une dimension pour appliquer le masque aux 3 canaux RGB
-   **Masque inversé** : Conserve le sujet (sans le fond)
-   **Masque normal** : Conserve le fond de remplacement

Cette étape prépare les images pour la composition finale.

In [ ]:
# Application des masques pour isoler les différentes parties
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
im = axs[0].imshow(just_do_it_crop)

# Application du masque inversé au sujet (garde le sujet, supprime le fond)
# expand_dims : ajoute une dimension pour que le masque 2D devienne 3D (compatible RGB)
just_do_it_masked = np.expand_dims(inverted_mask, axis=-1)*just_do_it_crop
im = axs[1].imshow(just_do_it_masked)

# Application du masque normal au fond de remplacement
fire_masked = np.expand_dims(mask, axis=-1)*fire_resize
im = axs[2].imshow(fire_masked)

fig.tight_layout()
plt.show()

---

## 6. Composition d'images : erreur classique avec le flou

Lors de la composition, il faut faire attention à l'ordre des opérations :
-   **Addition simple** : Combine les deux images masquées
-   **Normalisation** : Ramène les valeurs dans la plage 0-255
-   **Erreur avec gaussian_filter** : Appliquer le filtre directement sur l'image 3D floute les canaux RGB ensemble (incorrect)

Le deuxième résultat montre l'effet indésirable du floutage incorrect.

In [ ]:
# Composition des images avec et sans flou (approche incorrecte)
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

# Composition simple : addition des images masquées
blending = fire_masked + just_do_it_masked
blending = blending / blending.max()  # Normalisation
blending = (blending * 255).astype(np.uint8)
im = axs[0].imshow(blending)

# Erreur classique : le gaussian_filter affecte X/Y/RGB simultanément
# Cela crée un mélange indésirable entre les canaux de couleur
blending = gaussian_filter(fire_masked, sigma=2) + gaussian_filter(just_do_it_masked, sigma=2)
blending = blending / blending.max()
blending = (blending * 255).astype(np.uint8)
im = axs[1].imshow(blending)

fig.tight_layout()
plt.show()

---

## 7. Composition d'images : méthode correcte avec le flou

Pour appliquer correctement un filtre gaussien à une image RGB :
-   **Traiter chaque canal indépendamment** : Boucle sur les 3 canaux (R, G, B)
-   **Appliquer le filtre 2D** : gaussian_filter sur chaque canal séparément
-   **Normaliser individuellement** : Chaque canal est normalisé indépendamment

Cette approche préserve les couleurs correctes tout en lissant l'image.

In [ ]:
# Composition correcte des images avec flou par canal
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

# Composition simple (référence)
blending = fire_masked + just_do_it_masked
blending = blending / blending.max()
blending = (blending * 255).astype(np.uint8)
im = axs[0].imshow(blending)

# Méthode correcte : traiter chaque canal RGB indépendamment
blending = np.zeros((800,800,3))
for i in range(3):
    # Application du flou sur chaque canal séparément
    blending[:,:,i] = gaussian_filter(fire_masked[:,:,i], sigma=1) \
        + gaussian_filter(just_do_it_masked[:,:,i], sigma=1)
    # Normalisation de chaque canal
    blending[:,:,i] = gaussian_filter(blending[:,:,i] / blending[:,:,i].max(), sigma=1)

im = axs[1].imshow(blending)

fig.tight_layout()
plt.show()